In [ ]:
import os
import numpy as np
import pandas as pd
import qtconsole
import matplotlib
import matplotlib.pyplot as plt
import BeamDynamics as bd
try:
    import ROOT
except:
    print('Root framework not available.')

In [ ]:
%qtconsole

In [ ]:
%matplotlib inline
plt.rcParams['figure.figsize'] = [16, 9]
# %matplotlib notebook

# Root data structure from FCCee Target Tracking

## Open file

In [ ]:
runPath = bd.build_data_path('SimulationRuns/Geant4/000001')
rootFilePath = os.path.join(runPath, 'FCCeeTargetTracking.root')
outFile = ROOT.TFile.Open(rootFilePath, 'READ')

## Inspect structure

### File structure with ls() or GetListOfKeys()

In [ ]:
outFile.ls()

In [ ]:
treeList = outFile.GetListOfKeys()
for k in treeList:
    print(k)

In [ ]:
for k in treeList:
    print(k.GetName())

### Select TTree with Get()

In [ ]:
amorLeave = outFile.Get('amor_leave')
nAmorLeave = amorLeave.GetEntries()
print('Particles leaving the amorphous target: {:d}\n'.format(nAmorLeave))

### TTree strcuture with Print()

In [ ]:
amorLeave.Print()

### Default entry with Show()

In [ ]:
amorLeave.Show()

## Conversion to Python Pandas Data Frame

### Step 1: From Root TTree to R Data Frame

In [ ]:
amorLeaveRdf = ROOT.RDataFrame(amorLeave)

### Step 2: From R Data Frame to Dict with NumPy Arrays

In [ ]:
amorLeaveDict = amorLeaveRdf.AsNumpy()
print(amorLeaveDict.keys())
print([q.shape for q in amorLeaveDict.values()])

### Step 3: From NumPy Arrays to Pandas Data Frame

In [ ]:
amorLeaveMat = np.column_stack([q for q in amorLeaveDict.values()])
amorLeaveMat.shape
amorLeaveDf = pd.DataFrame(data=amorLeaveMat, columns=amorLeaveDict.keys())
amorLeaveDf.rename(columns={'e':'E'}, inplace=True)
amorLeaveDf['pdgId'] = amorLeaveDf['pdgId'].astype(int)
print(amorLeaveDf.columns)
print(amorLeaveDf.shape)

### Inspecting Pandas Data Frame

In [ ]:
amorLeaveDf.head()

In [ ]:
amorLeaveDf.tail()

In [ ]:
amorLeaveDf.describe()

### Selecting data in Pandas Data Frame

In [ ]:
positronId = -11
positronLeaveDf = amorLeaveDf[amorLeaveDf['pdgId']==positronId]
positronLeaveDf.head()

### (Deprecated method for conversion)

In [ ]:
amorLeaveMatDepr, amorLeaveLabelsDepr = amorLeave.AsMatrix(return_labels=True)
print(amorLeaveLabelsDepr)
print(amorLeaveMatDepr.shape)

## Conversion packed in a Python function

```python
bd.convert_fcceett_to_standard_df()
```

In [ ]:
sdfFilePath = os.path.join(runPath, 'FCCeeTargetTracking_amor_leave_StandardDf.txt')
try:
    distrFcceett = bd.load_standard_fwf(sdfFilePath)
except FileNotFoundError:
    distrFcceettAll = bd.convert_fcceett_to_standard_df(rootFilePath, pdgId=-11, saveStandardFwf=True)
    distrFcceett = distrFcceettAll['amor_leave']

<div class="alert alert-block alert-success">
The Python script<br />
GIT_PSIPositronProduction/Geant4/FCCeeTargetTrackingconvert_fcceett_to_standard_df.py<br />
simply calls the homonymous function and can be used to automatize the conversion, like e.g. in<br />
GIT_PSIPositronProduction/Geant4/FCCeeTargetTracking/run_Injector/run_Merlin.sh
</div>